# Quantum Phase Estimation - Maxime Nurwubusa

### Question 1

In [ ]:
!pip install wand
!apt install imagemagick
!pip install myqlm
!python -m qat.magics.install

import numpy as np
from scipy import linalg
import matplotlib.pylab as plt



from qat.lang.AQASM import Program, H,  AbstractGate, QRoutine, S, CNOT
from scipy import linalg # for linalg.expm, the matrix exponential.
from qat.qpus import get_default_qpu # numerical simulator for small quantum circuits.


prog = Program()
q = prog.qalloc(2)
prog.apply(H, q[0])
prog.apply(S.dag(), q[1])
prog.apply(CNOT, [q[0],q[1]])

def matrix(theta):
    X = np.array([[0,1],[1,0]])
    return linalg.expm(-1j * theta * X)

ham_x = AbstractGate("ham_X", [float], arity=1, matrix_generator=matrix)

prog.apply(ham_x(0.3), q[0])
prog.apply(H.ctrl(), q)
circ = prog.to_circ()
prog.apply(CNOT, q)
circ2 = prog.to_circ()


# TODO: put your implementation here. Take inspiration from the "minimal notebook" that was sent to you.

### Hamiltonian data

The purpose of the TP is to plot the “dissociation curve” of H2, for
various values of the Trotterization parameter p, which is done at the end
of the notebook.

This has already been done in the following document (Figure 3.a) : https://arxiv.org/abs/1512.06860.


The code below imports the data of Table I of https://arxiv.org/abs/1512.06860.


In [ ]:
#importing Hamiltonian data
import json 

with open('hamiltonian_data.json','r') as f:
    ham_data = json.load(f)
    
for coeffs in ham_data:
    print(coeffs)

### Question 2:
The code below implements QRoutines for each of the Hamiltonians we will need later.



In [ ]:
from qat.lang.AQASM import CNOT, RZ, RX, RY, S, I
from tp_library_pea import U_II, U_ZI, U_IZ, U_ZZ, U_XX, U_YY
# TODO: implement U_II, U_ZI, U_IZ, U_ZZ, U_XX, U_YY in the file tp_library_pea.py
# JUPYTER TRICK: you might have to kernel -> restart and run all to see the effect here of your code edition.

test = False
if test:
    prog = Program()
    q = prog.qalloc(2)
    prog.apply(U_YY(3.), q)
    circ = prog.to_circ()

    for op in circ.iterate_simple():
        print(op)

!python3 -m pytest tp_pea_question2_tests.py

### Question 3:

In [ ]:
from tp_library_pea import trotter_ham_simulation

!python3 -m pytest tp_pea_question3_tests.py

### Question 4: Iterative phase estimation



In [ ]:
from tp_library_pea import compute_phi_k, perfect_ham_simulation
# perfect_ham_simulation: ideal, non-Trotterized simulation.

!python3 -m pytest tp_pea_question4_tests.py

In [6]:
from qat.lang.AQASM import X

E_max = 3
E_min = -2
    
dt = (2 * np.pi) / float(E_max)

def phase(coeffs, trotterization=False, trotter_number=4, shift=-E_min, nBits = 10):
    """
    Given Hamiltonian coefficients, compute phi, s.t U|\psi\rangle = e^{-2i\pi\phi}|\psi\rangle
    
    Args:
        - coeffs: a dictionary of coefficients as extracted from the list of dictionaries loaded
        from hamiltonian_data.json
        - trotterization: Boolean flag specifying whether to use the Trotterized evolution or the
        ideal "cheat mode" which exponentiates the Hamiltonian.
        - trotter_number: the "p" controlling the degree of approximation of the Trotterization.
        - shift: the energy shift that we use to make sure that the phase we compute is 0 < phi < 1
        
    Returns:
        - phi, a real number that should fall between 0 and 1.
    """
    bits = {}

    for k in range(nBits, 0, -1):
        
        # CIRCUIT CREATION
        prog = Program()

        q = prog.qalloc(3)

        prog.apply(H, q[0])

        # ansatz preparation, we are lucky it is so simple.
        prog.apply(X, q[1])

        # Trotterization
        if trotterization:
            prog.apply(trotter_ham_simulation(coeffs, (2**(k-1)) * dt, trotter_number, shift).ctrl(), q)
        else:
            prog.apply(perfect_ham_simulation(coeffs, (2**(k-1)) * dt, shift).ctrl(), q)

        phi_k = compute_phi_k(bits, nBits, k) #I added k to the arguments

        # BEGIN IMPLEMENTATION. DO NOT MODIFY WHAT IS ABOVE.
        
        prog.apply(RZ(phi_k), q[0])
        prog.apply(H, q[0])


        
        # END IMPLEMENTATION. DO NOT MODIFY WHAT IS BELOW.
        
        circ = prog.to_circ()
                
        # CIRCUIT SIMULATION
        job = circ.to_job(qubits=[0])

        qpu = get_default_qpu()
        result = qpu.submit(job)

        # SELECTION OF MOST LIKELY RESULT 
        max_proba = -1
        max_state = -1
        for sample in result:
            if sample.probability > max_proba:
                max_proba = sample.probability
                max_state = sample.state.int
     
        bits[k] = max_state
          
    # recompute phi
    phi = 0
    for l in range(1,nBits+1,1):
        phi += float(bits[l])/float(2**l)
            
    return phi

### Question 4 - Dissociation curves

The code below uses the function above to compute phases for each values of R, convert them back to energies and plot the result for different Trotter number values (4 and 10). One can notice that p=10 gives a very good result, with respect to the document https://arxiv.org/abs/1512.06860.

In [ ]:
vals_perfect = []
vals_trotter_4 = []
vals_trotter_10 = []
Rs = []

shift = -E_min

for coeffs in ham_data:
    phi_perfect = phase(coeffs)
    phi_trotter_4 = phase(coeffs, trotterization=True, trotter_number=4)
    phi_trotter_10 = phase(coeffs, trotterization=True, trotter_number=10)

    # CONVERT PHASES BACK TO ENERGY
    E = ((2*np.pi)/dt)*phi_perfect - shift
    E_trotter_4 = ((2*np.pi)/dt)*phi_trotter_4 - shift 
    E_trotter_10 = ((2*np.pi)/dt)*phi_trotter_10 -shift

    
    print("R", coeffs['R'])
    Rs.append(coeffs['R'])
    
    vals_perfect.append(E)
    vals_trotter_4.append(E_trotter_4)
    vals_trotter_10.append(E_trotter_10)


In [ ]:
import matplotlib.pylab as plt

plt.plot(Rs, vals_perfect, label="perfect")
plt.plot(Rs, vals_trotter_4, label="p=4")
plt.plot(Rs, vals_trotter_10, label="p=10")
plt.legend()